In [ ]:
pip install numpy matplotlib pandas

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

a = 4
b = 6
c = 4
d = 4

def f(x):
    return (a * np.exp(-x)) + (b * np.sin(c * x)) - d

print("Fungsi Kelompok didefinisikan dengan parameter dari NPM anggota:")
print(f"Anggota 1: NPM ...54 -> a = {a}")
print(f"Anggota 2: NPM ...56 -> b = {b}")
print(f"Anggota 3: NPM ...74 -> c = {c}")
print(f"Hitung d: ({a} + {b} + {c}) mod 10 = {d}")
print("\nFungsi f(x) kelompok dalam bentuk matematika:")
print(f"f(x) = {a} * e^(-x) + {b} * sin({c} * x) - {d}")

In [ ]:
def lagrange_interpolation(x_points, y_points, x_test, verbose=False):
    n = len(x_points)
    L = np.zeros(n)
    y_interp = 0
    
    if verbose:
        print(f"\n--- Perhitungan Interpolasi Lagrange untuk x = {x_test} ---")
        print("Rumus: P(x) = Σ y_i * L_i(x), di mana L_i(x) = Π_{j≠i} (x - x_j) / (x_i - x_j)")
        print(f"Titik data: x = {[round(x, 1) for x in list(x_points)]}, y = {[round(y, 4) for y in list(y_points)]}")
    
    for i in range(n):
        L[i] = 1
        if verbose:
            print(f"\nMenghitung L_{i}({x_test}):")
        for j in range(n):
            if i != j:
                term = (x_test - x_points[j]) / (x_points[i] - x_points[j])
                L[i] *= term
                if verbose:
                    print(f"  L_{i} *= ({x_test} - {x_points[j]:.1f}) / ({x_points[i]:.1f} - {x_points[j]:.1f}) = {term:.6f} -> L_{i} = {L[i]:.6f}")
        contrib = y_points[i] * L[i]
        y_interp += contrib
        if verbose:
            print(f"  Kontribusi: y_{i} * L_{i} = {y_points[i]:.6f} * {L[i]:.6f} = {contrib:.6f}")
            print(f"  P(x) sekarang: {y_interp:.6f}")
        
    if verbose:
        print(f"\nHasil akhir: P({x_test}) = {y_interp:.6f}")
    return y_interp

def newton_forward_interpolation(x_points, y_points, x_test, verbose=False):
    n = len(x_points)
    coef_df = divided_diff_table(x_points, y_points)
    coef = coef_df.values
    
    if verbose:
        print(f"\n--- Perhitungan Interpolasi Newton Maju untuk x = {x_test} ---")
        print("Rumus: P(x) = f[x0] + (x - x0) * f[x0,x1] + (x - x0)(x - x1) * f[x0,x1,x2] + ...")
        print(f"Titik data: x = {[round(x, 1) for x in list(x_points)]}, y = {[round(y, 4) for y in list(y_points)]}")
        print("Koefisien diambil dari tabel selisih terbagi maju:")
        display(coef_df)

    p = coef[0][0]
    if verbose:
        print(f"P(x) = {p:.6f}")
    
    for i in range(1, n):
        term = coef[0][i]
        for j in range(i):
            term *= (x_test - x_points[j])
        p += term
        if verbose:
            print(f" + {coef[0][i]:.6f} * " + " * ".join([f"(x - {x_points[j]:.1f})" for j in range(i)]) + f" = {term:.6f} -> P(x) = {p:.6f}")
    
    if verbose:
        print(f"\nHasil akhir: P({x_test}) = {p:.6f}")
    return p

In [ ]:
x_data = np.array([0.0, 0.5, 1.0, 2.0])
y_data = f(x_data)

x_uji = 1.5 

y_hasil_lagrange = lagrange_interpolation(x_data, y_data, x_uji)
y_sejati = f(x_uji)

y_hasil_newton = newton_forward_interpolation(x_data, y_data, x_uji)

print("--- Detail Perhitungan Lagrange untuk x = 1.5 ---")
n = len(x_data)
L = np.zeros(n)
y_interp = 0
for i in range(n):
    L[i] = 1
    for j in range(n):
        if i != j:
            L[i] *= (x_uji - x_data[j]) / (x_data[i] - x_data[j])
    contrib = y_data[i] * L[i]
    y_interp += contrib
    print(f"L_{i} = {L[i]:.6f}, y_{i} = {y_data[i]:.6f}, Kontribusi = {contrib:.6f}")
print(f"Total P({x_uji}) = {y_interp:.6f}")

import pandas as pd

x_all = np.append(x_data, x_uji)
y_asli_all = f(x_all)
y_interp_lagrange = [lagrange_interpolation(x_data, y_data, x) for x in x_all]
y_interp_newton = [newton_forward_interpolation(x_data, y_data, x) for x in x_all]
error_lagrange = np.abs(y_asli_all - y_interp_lagrange)
error_newton = np.abs(y_asli_all - y_interp_newton)

df_hasil = pd.DataFrame({
    'x': x_all,
    'f(x) Asli': y_asli_all,
    'P(x) Lagrange': y_interp_lagrange,
    'Error Lagrange': error_lagrange,
    'P(x) Newton Maju': y_interp_newton,
    'Error Newton': error_newton
})

print("\n--- Tabel Hasil Perhitungan Interpolasi ---")
display(df_hasil)

In [ ]:
def divided_diff_table(x, y):
    n = len(y)
    coef = np.zeros([n, n])
    coef[:,0] = y
    
    for j in range(1,n):
        for i in range(n-j):
            coef[i][j] = (coef[i+1][j-1] - coef[i][j-1]) / (x[i+j]-x[i])

    df = pd.DataFrame(coef)
    columns = [f'f[x{i}]' if i==0 else f'Orde {i}' for i in range(n)]
    df.columns = columns
    df.index = x
    return df

print("--- Tabel Selisih Terbagi Maju ---")
print("Ini adalah selisih terbagi maju (forward divided differences), digunakan untuk interpolasi Newton maju.")
print("Rumus: f[x_i, x_{i+1}, ..., x_{i+k}] = (f[x_{i+1}, ..., x_{i+k}] - f[x_i, ..., x_{i+k-1}]) / (x_{i+k} - x_i)")
df_table = divided_diff_table(x_data, y_data)
display(df_table)

coef = df_table.values
print("\n--- Polinom Interpolasi Newton Maju P(x) ---")
p_str = f"{coef[0][0]:.6f}"
for i in range(1, len(coef)):
    term = f" + {coef[0][i]:.6f}"
    for j in range(i):
        term += f"*(x - {x_data[j]:.1f})"
    p_str += term
print(f"P(x) = {p_str}")
print("Ini adalah polinom derajat 3 yang melewati semua titik data.")

In [ ]:
x_plot = np.linspace(min(x_data)-0.5, max(x_data)+0.5, 100)
y_plot_asli = f(x_plot)
y_plot_lagrange = [lagrange_interpolation(x_data, y_data, xp) for xp in x_plot]
y_plot_newton = [newton_forward_interpolation(x_data, y_data, xp) for xp in x_plot]

plt.figure(figsize=(10, 6))

plt.plot(x_plot, y_plot_asli, label=f'Fungsi Asli $f(x)={a}e^{{-x}} + {b}\\sin({c}x) - {d}$', color='blue')

plt.plot(x_plot, y_plot_lagrange, label='Polinom Interpolasi Lagrange $P(x)$', color='red', linestyle='--')

plt.plot(x_plot, y_plot_newton, label='Polinom Interpolasi Newton Maju $P(x)$', color='orange', linestyle='-.')

plt.scatter(x_data, y_data, color='black', zorder=5, label='Titik Data (4 titik)')

plt.scatter(x_uji, y_hasil_lagrange, color='green', marker='X', s=100, label=f'Titik Uji x={x_uji} (Lagrange)')
plt.scatter(x_uji, y_hasil_newton, color='purple', marker='s', s=100, label=f'Titik Uji x={x_uji} (Newton)')

plt.title(f"Interpolasi: Kelompok 4, 6, 4")
plt.xlabel("x")
plt.ylabel("f(x)")
plt.legend()
plt.grid(True)
plt.savefig('grafik_interpolasi.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
error_abs = abs(y_sejati - y_hasil_lagrange)

print("--- Analisis Error ---")
print(f"Nilai Sejati      : {y_sejati:.8f}")
print(f"Nilai Interpolasi : {y_hasil_lagrange:.8f}")
print(f"Error Mutlak      : {error_abs:.8f}")
print("Kesimpulan: Semakin kecil error, semakin akurat polinom interpolasi di titik tersebut.")